# Introduzione

Obiettivo di questo notebook: testare il training degli algoritmi Implicit Q-Learning, Conservative Q-Learning, Behavior Cloning, TD3 con Behavior Cloning sull dataset `D4RL/pen/expert-v2`.

Il task "pen" richiede a una mano robotica (Adroit Hand) di manipolare una penna per portarla in una certa posizione nello spazio.

# Caricamento del dataset

In [42]:
import minari
import time
import numpy as np
from d3rlpy.algos import IQLConfig, CQLConfig, BCConfig, TD3PlusBCConfig
from d3rlpy.datasets import MDPDataset
from d3rlpy.constants import ActionSpace
from d3rlpy.metrics import EnvironmentEvaluator

In [2]:
dataset = minari.load_dataset("D4RL/pen/expert-v2")

In [3]:
print("Episodi totali:", dataset.total_episodes)
print("Spazio osservazioni:", dataset.observation_space)
print("Spazio azioni:", dataset.action_space)

Episodi totali: 4958
Spazio osservazioni: Box(-inf, inf, (45,), float64)
Spazio azioni: Box(-1.0, 1.0, (24,), float32)


In [4]:
episode = next(dataset.iterate_episodes())
print(episode)

#print(f"Osservazioni: \n{episode.observations[0]}")
#print(f"Actions: \n{episode.actions[0]}")
#print(f"Rewards: \n{episode.rewards[0]}")
#print(f"Terminations: \n{episode.terminations[0]}")

EpisodeData(id=0, total_steps=100, observations=ndarray of shape (101, 45) and dtype float64, actions=ndarray of shape (100, 24) and dtype float32, rewards=ndarray of 100 floats, terminations=ndarray of 100 bools, truncations=ndarray of 100 bools, infos=dict with the following keys: ['success'])


The task to be completed consists on repositioning the blue pen to match the orientation of the green target. The base of the hand is fixed. The target is also randomized to cover all configurations. The task will be considered successful when the orientations match within tolerance

# Preparazione dataset

d3rlpy si aspetta che il dataset sia composto da transizioni, in cui ogni elemento contiene uno stato, un’azione, una ricompensa, lo stato successivo e un flag terminale, tutti allineati in modo che lo stato e l’azione alla posizione i corrispondano alla transizione verso lo stato alla posizione i+1. A tal fine, la libreria mette a disposizione la classe MDPDataset, che consente di creare facilmente un oggetto dataset nel formato richiesto. All'interno del dataset non c'è la distinzione in episodi, tutti gli step sono uniti in un unico array.

In [5]:
observations = []
actions = []
rewards = []
terminals = []

for episode in dataset.iterate_episodes():
    # si rimuove l'ultimo elemento, in quanto non ha una successiva azione associata
    obs = episode.observations[:-1]
    actions_ep = episode.actions
    rewards_ep = episode.rewards
    dones = np.array(episode.terminations) | np.array(episode.truncations)

    observations.append(obs)
    actions.append(actions_ep)
    rewards.append(rewards_ep)
    terminals.append(dones)

# ora observations è un array di 4958 array (episodi) di 100 array circa (step) di array (osservazioni). stesso discorso per gli altri

# si uniscono gli array in modo da avere, per ogni step del dataset osservazioni, azione, reward, terminali
observations = np.concatenate(observations)
actions = np.concatenate(actions)
rewards = np.concatenate(rewards)
terminals = np.concatenate(terminals)

# ora observations è un array di 499206 (step in tutto il dataset) di array (osservazioni) . stesso discorso per gli altri
print(observations.shape)
print(actions.shape)
print(rewards.shape)
print(terminals.shape)

d3_dataset = MDPDataset(observations, actions, rewards, terminals, action_space = ActionSpace.CONTINUOUS)

(499206, 45)
(499206, 24)
(499206,)
(499206,)
2025-04-07 09:34.43 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-04-07 09:34.43 [info     ] Action size has been automatically determined. action_size=24


# Implicit Q-Learning

In [6]:
iql = IQLConfig().create(device="cpu")

In [7]:
iql.build_with_dataset(d3_dataset)

In [9]:
env = dataset.recover_environment()

iql.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

2025-04-07 09:35.04 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=24)
2025-04-07 09:35.04 [warning  ] Skip building models since they're already built.
2025-04-07 09:35.04 [info     ] Directory is created at d3rlpy_logs/IQL_20250407093504
2025-04-07 09:35.04 [info     ] Parameters                     params={'observation_shape': [45], 'action_size': 24, 'config': {'type': 'iql', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'actor_optim_factory': {'type': 'adam', 'params

Epoch 1/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:35.10 [info     ] IQL_20250407093504: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0015404837131500243, 'time_algorithm_update': 0.004327301740646362, 'critic_loss': 2675.132789794922, 'q_loss': 2659.8955614318847, 'v_loss': 15.23722476863861, 'actor_loss': 58.79665441417694, 'time_step': 0.005905513525009155, 'env': 2895.1029235642613} step=1000
2025-04-07 09:35.10 [info     ] Model parameters are saved to d3rlpy_logs/IQL_20250407093504/model_1000.d3


Epoch 2/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:35.17 [info     ] IQL_20250407093504: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.0015875694751739501, 'time_algorithm_update': 0.004651618719100952, 'critic_loss': 6148.825401062012, 'q_loss': 6121.134932830811, 'v_loss': 27.690466495513917, 'actor_loss': 47.209079043388364, 'time_step': 0.006278772592544555, 'env': 3243.914709515445} step=2000
2025-04-07 09:35.17 [info     ] Model parameters are saved to d3rlpy_logs/IQL_20250407093504/model_2000.d3


...

Epoch 9/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:36.02 [info     ] IQL_20250407093504: epoch=9 step=9000 epoch=9 metrics={'time_sample_batch': 0.0015833544731140136, 'time_algorithm_update': 0.004581630229949951, 'critic_loss': 42668.74803686523, 'q_loss': 42504.558054931644, 'v_loss': 164.1900009765625, 'actor_loss': 32.31873790705204, 'time_step': 0.006203950166702271, 'env': 3990.6853455955543} step=9000
2025-04-07 09:36.02 [info     ] Model parameters are saved to d3rlpy_logs/IQL_20250407093504/model_9000.d3


Epoch 10/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:36.09 [info     ] IQL_20250407093504: epoch=10 step=10000 epoch=10 metrics={'time_sample_batch': 0.0015127980709075928, 'time_algorithm_update': 0.004278781175613403, 'critic_loss': 45794.06400708008, 'q_loss': 45618.16446240235, 'v_loss': 175.89949571990968, 'actor_loss': 32.171769808888435, 'time_step': 0.005828266382217407, 'env': 2531.518444096145} step=10000
2025-04-07 09:36.09 [info     ] Model parameters are saved to d3rlpy_logs/IQL_20250407093504/model_10000.d3


[(1,
  {'time_sample_batch': 0.0015404837131500243,
   'time_algorithm_update': 0.004327301740646362,
   'critic_loss': 2675.132789794922,
   'q_loss': 2659.8955614318847,
   'v_loss': 15.23722476863861,
   'actor_loss': 58.79665441417694,
   'time_step': 0.005905513525009155,
   'env': 2895.1029235642613}),
 (2,
  {'time_sample_batch': 0.0015875694751739501,
   'time_algorithm_update': 0.004651618719100952,
   'critic_loss': 6148.825401062012,
   'q_loss': 6121.134932830811,
   'v_loss': 27.690466495513917,
   'actor_loss': 47.209079043388364,
   'time_step': 0.006278772592544555,
   'env': 3243.914709515445}),
 (3,
  {'time_sample_batch': 0.0015845096111297607,
   'time_algorithm_update': 0.004515581607818604,
   'critic_loss': 10893.418867370605,
   'q_loss': 10849.978895690918,
   'v_loss': 43.439971202850344,
   'actor_loss': 41.11088452291489,
   'time_step': 0.0061383905410766606,
   'env': 3545.555285391757}),
 (4,
  {'time_sample_batch': 0.0015312747955322265,
   'time_algorit

In [25]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = iql.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated:
        break

env.close()
print(f"Reward totale: {total_reward}")

Reward totale: 5152.226734340886


# Conservative Q-Learning

In [14]:
cql = CQLConfig().create(device="cpu")

In [16]:
cql.build_with_dataset(d3_dataset)

In [17]:
env = dataset.recover_environment()

cql.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

2025-04-07 09:39.05 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=24)
2025-04-07 09:39.05 [warning  ] Skip building models since they're already built.
2025-04-07 09:39.05 [info     ] Directory is created at d3rlpy_logs/CQL_20250407093905
2025-04-07 09:39.05 [info     ] Parameters                     params={'observation_shape': [45], 'action_size': 24, 'config': {'type': 'cql', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0001, 'critic_learning_rate': 0.0003, 'temp_learning_rate': 0.0001, 'alpha_learning_r

Epoch 1/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:39.41 [info     ] CQL_20250407093905: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0018825025558471679, 'time_algorithm_update': 0.033327054500579836, 'critic_loss': 850.7936302185059, 'conservative_loss': 3.736338350892067, 'alpha': 1.0130486673116683, 'actor_loss': -91.61962862110138, 'temp': 0.9556084146499634, 'temp_loss': 29.52512632369995, 'time_step': 0.0352595739364624, 'env': 185.83549634159868} step=1000
2025-04-07 09:39.41 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20250407093905/model_1000.d3


Epoch 2/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:40.20 [info     ] CQL_20250407093905: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.0019532697200775146, 'time_algorithm_update': 0.03672051596641541, 'critic_loss': 2269.0222620391846, 'conservative_loss': -39.814375631332396, 'alpha': 0.9383266568779945, 'actor_loss': -244.5562688140869, 'temp': 0.8765679097175598, 'temp_loss': 21.79221344947815, 'time_step': 0.03872582507133484, 'env': 241.60821961057314} step=2000
2025-04-07 09:40.20 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20250407093905/model_2000.d3


...

Epoch 9/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:44.25 [info     ] CQL_20250407093905: epoch=9 step=9000 epoch=9 metrics={'time_sample_batch': 0.0019985873699188235, 'time_algorithm_update': 0.03291210985183716, 'critic_loss': 71410.32231298828, 'conservative_loss': 598.3275304870606, 'alpha': 1.5261711584329605, 'actor_loss': -1784.3933461914062, 'temp': 0.5873868223428726, 'temp_loss': -0.08099927717074752, 'time_step': 0.03496393513679504, 'env': 8.080350396701569} step=9000
2025-04-07 09:44.25 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20250407093905/model_9000.d3


Epoch 10/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:45.00 [info     ] CQL_20250407093905: epoch=10 step=10000 epoch=10 metrics={'time_sample_batch': 0.0019095804691314697, 'time_algorithm_update': 0.03276324963569641, 'critic_loss': 93127.96612402344, 'conservative_loss': 906.7369437255859, 'alpha': 1.748260232925415, 'actor_loss': -2063.5706540527344, 'temp': 0.5994024122357369, 'temp_loss': -0.845141943179071, 'time_step': 0.034725390672683716, 'env': 30.865614627095464} step=10000
2025-04-07 09:45.00 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20250407093905/model_10000.d3


[(1,
  {'time_sample_batch': 0.0018825025558471679,
   'time_algorithm_update': 0.033327054500579836,
   'critic_loss': 850.7936302185059,
   'conservative_loss': 3.736338350892067,
   'alpha': 1.0130486673116683,
   'actor_loss': -91.61962862110138,
   'temp': 0.9556084146499634,
   'temp_loss': 29.52512632369995,
   'time_step': 0.0352595739364624,
   'env': 185.83549634159868}),
 (2,
  {'time_sample_batch': 0.0019532697200775146,
   'time_algorithm_update': 0.03672051596641541,
   'critic_loss': 2269.0222620391846,
   'conservative_loss': -39.814375631332396,
   'alpha': 0.9383266568779945,
   'actor_loss': -244.5562688140869,
   'temp': 0.8765679097175598,
   'temp_loss': 21.79221344947815,
   'time_step': 0.03872582507133484,
   'env': 241.60821961057314}),
 (3,
  {'time_sample_batch': 0.0017448365688323975,
   'time_algorithm_update': 0.031003796339035033,
   'critic_loss': 6058.591243255615,
   'conservative_loss': -31.02224608898163,
   'alpha': 0.8535002691745758,
   'actor_lo

In [30]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = cql.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated:
        break

env.close()
print(f"Reward totale: {total_reward}")

Reward totale: -4405.210054516312


# Behavior Cloning

In [34]:
bc = BCConfig().create(device="cpu")

In [35]:
bc.build_with_dataset(d3_dataset)

In [36]:
env = dataset.recover_environment()

bc.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

2025-04-07 09:50.31 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=24)
2025-04-07 09:50.31 [warning  ] Skip building models since they're already built.
2025-04-07 09:50.31 [info     ] Directory is created at d3rlpy_logs/BC_20250407095031
2025-04-07 09:50.31 [info     ] Parameters                     params={'observation_shape': [45], 'action_size': 24, 'config': {'type': 'bc', 'params': {'batch_size': 100, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'learning_rate': 0.001, 'policy_type': 'deterministic', 'optim_factory': {'type': 'adam', 'params': {'clip_grad_

Epoch 1/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:50.33 [info     ] BC_20250407095031: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0006606235504150391, 'time_algorithm_update': 0.000628838062286377, 'loss': 0.10639423998445273, 'time_step': 0.0013068647384643554, 'env': 2617.8516269407746} step=1000
2025-04-07 09:50.33 [info     ] Model parameters are saved to d3rlpy_logs/BC_20250407095031/model_1000.d3


Epoch 2/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:50.34 [info     ] BC_20250407095031: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.0005909712314605713, 'time_algorithm_update': 0.0005823712348937988, 'loss': 0.09810689052194357, 'time_step': 0.0011873888969421388, 'env': 3518.544428844261} step=2000
2025-04-07 09:50.34 [info     ] Model parameters are saved to d3rlpy_logs/BC_20250407095031/model_2000.d3


...

Epoch 9/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:50.46 [info     ] BC_20250407095031: epoch=9 step=9000 epoch=9 metrics={'time_sample_batch': 0.0005875816345214844, 'time_algorithm_update': 0.000578500509262085, 'loss': 0.09610408852249384, 'time_step': 0.0011793177127838136, 'env': 2316.8861195542713} step=9000
2025-04-07 09:50.46 [info     ] Model parameters are saved to d3rlpy_logs/BC_20250407095031/model_9000.d3


Epoch 10/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:50.47 [info     ] BC_20250407095031: epoch=10 step=10000 epoch=10 metrics={'time_sample_batch': 0.0006136603355407715, 'time_algorithm_update': 0.0006180744171142578, 'loss': 0.09613627146184445, 'time_step': 0.0012479488849639893, 'env': 2495.5037871943528} step=10000
2025-04-07 09:50.47 [info     ] Model parameters are saved to d3rlpy_logs/BC_20250407095031/model_10000.d3


[(1,
  {'time_sample_batch': 0.0006606235504150391,
   'time_algorithm_update': 0.000628838062286377,
   'loss': 0.10639423998445273,
   'time_step': 0.0013068647384643554,
   'env': 2617.8516269407746}),
 (2,
  {'time_sample_batch': 0.0005909712314605713,
   'time_algorithm_update': 0.0005823712348937988,
   'loss': 0.09810689052194357,
   'time_step': 0.0011873888969421388,
   'env': 3518.544428844261}),
 (3,
  {'time_sample_batch': 0.0005932431221008301,
   'time_algorithm_update': 0.0006242425441741943,
   'loss': 0.09756314815580845,
   'time_step': 0.001231632947921753,
   'env': 2028.498268216287}),
 (4,
  {'time_sample_batch': 0.000613410472869873,
   'time_algorithm_update': 0.0006431434154510499,
   'loss': 0.09678063146024943,
   'time_step': 0.0012755467891693115,
   'env': 3487.7231959085757}),
 (5,
  {'time_sample_batch': 0.0005931143760681153,
   'time_algorithm_update': 0.000586411714553833,
   'loss': 0.09684043920785189,
   'time_step': 0.0011939258575439453,
   'env'

In [41]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = bc.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated:
        break

env.close()
print(f"Reward totale: {total_reward}")

Reward totale: 59847.596286878754


# TD3 + BC

In [43]:
td3bc = TD3PlusBCConfig().create(device="cpu")

In [44]:
td3bc.build_with_dataset(d3_dataset)

In [45]:
env = dataset.recover_environment()

td3bc.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

2025-04-07 09:54.34 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=24)
2025-04-07 09:54.34 [warning  ] Skip building models since they're already built.
2025-04-07 09:54.34 [info     ] Directory is created at d3rlpy_logs/TD3PlusBC_20250407095434
2025-04-07 09:54.34 [info     ] Parameters                     params={'observation_shape': [45], 'action_size': 24, 'config': {'type': 'td3_plus_bc', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'actor_optim_factory': {'type': '

Epoch 1/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:54.39 [info     ] TD3PlusBC_20250407095434: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0016341910362243652, 'time_algorithm_update': 0.003619870901107788, 'critic_loss': 677.8931263122558, 'actor_loss': -2.057920249223709, 'bc_loss': 0.44182866632938383, 'time_step': 0.005290372610092163, 'env': 25.081717289435858} step=1000
2025-04-07 09:54.39 [info     ] Model parameters are saved to d3rlpy_logs/TD3PlusBC_20250407095434/model_1000.d3


Epoch 2/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:54.46 [info     ] TD3PlusBC_20250407095434: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.0016969678401947022, 'time_algorithm_update': 0.004318328142166138, 'critic_loss': 812.1644007720947, 'actor_loss': -2.307586257457733, 'bc_loss': 0.18955203637480736, 'time_step': 0.0060540554523468015, 'env': 254.08299282699954} step=2000
2025-04-07 09:54.46 [info     ] Model parameters are saved to d3rlpy_logs/TD3PlusBC_20250407095434/model_2000.d3


...

Epoch 9/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:55.28 [info     ] TD3PlusBC_20250407095434: epoch=9 step=9000 epoch=9 metrics={'time_sample_batch': 0.0016840426921844482, 'time_algorithm_update': 0.004356842279434204, 'critic_loss': 20406.089936340333, 'actor_loss': -2.3565680599212646, 'bc_loss': 0.14288178727030754, 'time_step': 0.006075936317443848, 'env': 212.47088908348474} step=9000
2025-04-07 09:55.28 [info     ] Model parameters are saved to d3rlpy_logs/TD3PlusBC_20250407095434/model_9000.d3


Epoch 10/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-07 09:55.34 [info     ] TD3PlusBC_20250407095434: epoch=10 step=10000 epoch=10 metrics={'time_sample_batch': 0.001544571876525879, 'time_algorithm_update': 0.003459683656692505, 'critic_loss': 25075.283450073242, 'actor_loss': -2.3554778652191164, 'bc_loss': 0.14416065487265586, 'time_step': 0.005035207986831665, 'env': 856.5322604740846} step=10000
2025-04-07 09:55.34 [info     ] Model parameters are saved to d3rlpy_logs/TD3PlusBC_20250407095434/model_10000.d3


[(1,
  {'time_sample_batch': 0.0016341910362243652,
   'time_algorithm_update': 0.003619870901107788,
   'critic_loss': 677.8931263122558,
   'actor_loss': -2.057920249223709,
   'bc_loss': 0.44182866632938383,
   'time_step': 0.005290372610092163,
   'env': 25.081717289435858}),
 (2,
  {'time_sample_batch': 0.0016969678401947022,
   'time_algorithm_update': 0.004318328142166138,
   'critic_loss': 812.1644007720947,
   'actor_loss': -2.307586257457733,
   'bc_loss': 0.18955203637480736,
   'time_step': 0.0060540554523468015,
   'env': 254.08299282699954}),
 (3,
  {'time_sample_batch': 0.0016006903648376465,
   'time_algorithm_update': 0.0037991657257080078,
   'critic_loss': 1875.3501327514648,
   'actor_loss': -2.339436363220215,
   'bc_loss': 0.1587618647515774,
   'time_step': 0.0054336261749267575,
   'env': 517.566570717139}),
 (4,
  {'time_sample_batch': 0.0015545170307159424,
   'time_algorithm_update': 0.0035663602352142333,
   'critic_loss': 3618.996870025635,
   'actor_loss':

In [50]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = td3bc.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated:
        break

env.close()
print(f"Reward totale: {total_reward}")

Reward totale: 866.3860965817254
